In [1]:
#%% library import
import numpy as np
import pandas as pd
import networkx as nx
import torch as tc
import torch
import pprint
import pickle
import time

from torch.autograd import Variable
from sklearn.utils import shuffle
from torch import nn
from torch import optim
from torch.nn import functional as F
from torch.utils.data import DataLoader
from functools import partial
from bayes_opt import BayesianOptimization


In [2]:
#%% Load dataset and cuda
dataset = pd.read_csv("datasets/KIBA.csv")
datalen = len(dataset)
cuda = tc.device('cuda')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(0)/1024**3,1), 'GB')

Using device: cuda
GeForce RTX 2080 Ti
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [3]:
#%% protein-ligand-kiba split
protein = dataset.loc[:, "uniprotID"]    # 5
ligand = dataset.loc[:, "chemblID"]
kiba = list(dataset.loc[:, 'KIBA'])
del dataset


In [4]:
#%% protein sequence load
f = open('datasets/dictionaries/prt_tfidf.txt', 'rb')
tfidf_dic = pickle.load(f)
f.close()

tfidf = np.zeros((datalen, 400))
for i, s in enumerate(protein):
    tfidf[i] = tfidf_dic[s]

In [5]:
#%% protein sequence load
f = open('datasets/dictionaries/prt_lstm.txt', 'rb')
seq_voc, _ = pickle.load(f)
f.close()

sequence = np.zeros((datalen, 4128))
for i, s in enumerate(protein):
    sequence[i] = seq_voc[s]

sequence = sequence[:, :1400]

In [6]:
#%% ligand smiles load
f = open('datasets/dictionaries/lgn_smiecoding.txt', 'rb')
smi_dic = pickle.load(f)
f.close()

smileseq = np.zeros((datalen, 590))
for i, e in enumerate(ligand):
    smileseq[i] = smi_dic[e]

smileseq = smileseq[:, :100]


In [7]:
#%% dataset zip
revised_dataset = list(zip(tfidf, sequence, smileseq, kiba))
shuffled_dataset = shuffle(revised_dataset); del revised_dataset
trainset = shuffled_dataset[:int((9/10)*datalen)]
validset = shuffled_dataset[int((9/10)*datalen):]

In [8]:
#%% Make collate func.
def collate(samples):
    tfidf, sequence, smileseq, labels = map(list, zip(*samples))
    return tc.tensor(tfidf, dtype=tc.float).cuda(), tc.LongTensor(sequence).cuda(), tc.LongTensor(smileseq).cuda(), tc.tensor(labels).cuda()

In [9]:
#%% learning module 선언
class Regressor(nn.Module):
    def __init__(self):
        super(Regressor, self).__init__()    # method 상속받고 __init__()은 여기서 하겠다.
        
        self.prt_cv1dlayers0 = nn.Sequential(
                        nn.Conv1d(1, 1, kernel_size = 32),
                        nn.BatchNorm1d(num_features = 1),
                        nn.ReLU()
                        )
        
        self.fc = nn.Linear(369, 256, F.relu)
        
        self.prt_emlayer = nn.Embedding(21, 10)
        
        self.prt_cv1dlayers1 = nn.Sequential(
                        nn.Conv1d(10, 32, kernel_size = 4),
                        nn.BatchNorm1d(num_features = 32),
                        nn.ReLU(),
                        nn.Conv1d(32, 64, kernel_size = 8),
                        nn.BatchNorm1d(num_features = 64),
                        nn.ReLU(),
                        nn.Conv1d(64, 96, kernel_size = 12),
                        nn.BatchNorm1d(num_features = 96),
                        nn.ReLU(),
                        nn.MaxPool1d(kernel_size=1379)
                        )
        
        ######################################################################
        ######################################################################
        self.lgn_emlayer = nn.Embedding(64, 10)
        
        self.lgn_cv1dlayers = nn.Sequential(
                        nn.Conv1d(10, 32, kernel_size = 4),
                        nn.BatchNorm1d(num_features = 32),
                        nn.ReLU(),
                        nn.Conv1d(32, 64, kernel_size = 6),
                        nn.BatchNorm1d(num_features = 64),
                        nn.ReLU(),
                        nn.Conv1d(64, 96, kernel_size = 8),
                        nn.BatchNorm1d(num_features = 96),
                        nn.ReLU(),
                        nn.MaxPool1d(kernel_size = 85)
                        )
        
        
        self.mlplayers = nn.Sequential(
                        nn.Linear(448, 1024),
                        nn.BatchNorm1d(1024),
                        nn.ReLU(),
                        nn.Dropout(0.1),
                        nn.Linear(1024, 1024),
                        nn.BatchNorm1d(1024),
                        nn.ReLU(),
                        nn.Dropout(0.1),
                        nn.Linear(1024, 1024),
                        nn.BatchNorm1d(1024),
                        nn.ReLU(),
                        nn.Dropout(0.1),
                        nn.Linear(1024, 512),
                        nn.BatchNorm1d(512),
                        nn.ReLU(),
                        nn.Dropout(0.1),
                        nn.Linear(512, 512),
                        nn.BatchNorm1d(512),
                        nn.ReLU()
                        )

        self.regress = nn.Linear(512, 1)    # regression

    def forward(self, tfidf, prt_seq, lgn_seq):
        p0 = tfidf.unsqueeze(1)
        p0 = self.prt_cv1dlayers0(p0)
        p0 = p0.squeeze()
        p0 = self.fc(p0)
        
        p1 = self.prt_emlayer(prt_seq)
        p1 = p1.permute(0, 2, 1)
        p1 = self.prt_cv1dlayers1(p1)
        p1 = p1.squeeze()
        
        l = self.lgn_emlayer(lgn_seq)
        l = l.permute(0, 2, 1)
        l = self.lgn_cv1dlayers(l)
        l = l.squeeze()
        
        cat = tc.cat((p0, p1, l), axis=1).cuda()
        out = self.mlplayers(cat)
        out = out.squeeze()
        
        return self.regress(out).cuda()

In [10]:
#%% Set hyperparameter
hp_d = {}

# FIXME: 학습 관련 하이퍼파라미터
hp_d['batch_size'] = 512
hp_d['num_epochs'] = 100

hp_d['init_learning_rate'] = 0
hp_d['eps'] = 0
hp_d['weight_decay'] = 0

In [11]:
#%% training and validation
tr_data_loader = DataLoader(trainset, batch_size=hp_d['batch_size'], shuffle=False, collate_fn=collate)
va_data_loader = DataLoader(validset, batch_size=hp_d['batch_size'], shuffle=False, collate_fn=collate)

print('tr_var:', np.var(np.array([s[3] for s in trainset])))
print('va_var:', np.var(np.array([s[3] for s in validset])))

def train_and_validate(init_learning_rate_log, weight_decay_log, eps):
    
    model = Regressor().to(torch.device('cuda:0'))
    loss_func = nn.MSELoss(reduction='mean').cuda()
    
    hp_d['init_learning_rate'] = 10**init_learning_rate_log
    hp_d['weight_decay'] = 10**weight_decay_log
    hp_d['eps'] = 10**eps
    
    optimizer = optim.Adam(model.parameters(), lr=hp_d['init_learning_rate'], 
        weight_decay=hp_d['weight_decay'], eps=hp_d['eps'])

#     optimizer = optim.Adam(model.parameters(), lr=0.001)

    tr_epoch_losses = []
    va_epoch_losses = []

    for epoch in range(hp_d['num_epochs']):                             #!! epoch-loop
        # training session
        model.train()
        tr_epoch_loss = 0

        for iter, (tfidf, seq, smi, label) in enumerate(tr_data_loader):       #!! batch-loop
            prediction = model(tfidf, seq, smi).view(-1).cuda()
            loss = loss_func(prediction, label).cuda()

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            tr_epoch_loss += loss.detach().item()

        tr_epoch_loss /= (iter + 1)
#         print('Training epoch {}, loss {:.4f}'.format(epoch, tr_epoch_loss))
        tr_epoch_losses.append(tr_epoch_loss)

    # ===========================================================================
        # validation session
        model.eval()
        va_epoch_loss = 0

        for iter, (tfidf, seq, smi, label) in enumerate(va_data_loader):  # batch-loop
            prediction = model(tfidf, seq, smi).view(-1).cuda()
            loss = loss_func(prediction, label).cuda()

            va_epoch_loss += loss.detach().item()

        va_epoch_loss /= (iter + 1)
#         print('Validation epoch {}, loss {:.4f}'.format(epoch, va_epoch_loss))
        va_epoch_losses.append(va_epoch_loss)
    
    return -min(va_epoch_losses)

tr_var: 0.7011378689884774
va_var: 0.694197146323285


In [13]:
bayes_optimizer = BayesianOptimization(
    f=train_and_validate,
    pbounds={
        'init_learning_rate_log': (-5, -1),      # FIXME
        'weight_decay_log': (-6.5, -3.5),            # FIXME
        'eps': (-8.5, -7)                          # FIXME
    },
    random_state=0,
    verbose=2
)

bayes_optimizer.maximize(init_points=3, n_iter=60, acq='ei', xi=0.01)    # FIXME

for i, res in enumerate(bayes_optimizer.res):
    print('Iteration {}: \n\t{}'.format(i, res))
print('Final result: ', bayes_optimizer.max)

|   iter    |  target   |    eps    | init_l... | weight... |
-------------------------------------------------------------
|  1        | -0.2021   | -7.677    | -2.139    | -4.692    |
|  2        | -0.1607   | -7.683    | -3.305    | -4.562    |
|  3        | -0.4284   | -7.844    | -1.433    | -3.609    |
|  4        | -0.2269   | -7.0      | -5.0      | -6.5      |
|  5        | -0.1538   | -8.5      | -2.644    | -6.5      |
|  6        | -0.165    | -7.0      | -2.052    | -6.5      |
|  7        | -0.2185   | -8.5      | -5.0      | -4.858    |
|  8        | -0.153    | -7.32     | -3.163    | -5.639    |
|  9        | -0.219    | -7.0      | -5.0      | -3.5      |
|  10       | -0.2452   | -8.5      | -1.0      | -6.5      |
|  11       | -0.1636   | -8.5      | -2.963    | -5.402    |
|  12       | -0.1663   | -7.0      | -3.134    | -4.853    |
|  13       | -0.1571   | -7.774    | -2.598    | -6.381    |


KeyboardInterrupt: 

In [14]:
#%%
np.save('ModifiedDeepDTA_v8_1_va_losses(optim)', bayes_optimizer.res)

In [3]:
0.25 + 0.5 == 0.75

True